# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

import pickle

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages', engine)

# fetch the message feature as input
X = df['message']
# use all other features as labels, while dropping 'orginal' (ie. message in native language), and 'genre'
Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)

In [ ]:
X.head()

### 2. Write a tokenization function to process your text data

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def tokenize(text: str) -> list:
    words = [word for word in word_tokenize(text.lower()) if word.isalnum()]
    return [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]

In [ ]:
tokenize(X[3])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
rfc = RandomForestClassifier(random_state=42, min_samples_split=7)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(estimator=rfc, n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

for i, column in enumerate(y_test):
    print(column)
    print(classification_report(y_test[column], y_pred[:, i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
rfc = RandomForestClassifier(random_state=42)
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('mclf', MultiOutputClassifier(estimator=rfc, n_jobs=-1))
])

parameters = {
    #'mclf__estimator__n_estimators': [100, 110],
    #'mclf__estimator__min_samples_split': [6, 8]
    'mclf__estimator__n_estimators': [100],
    'mclf__estimator__max_depth': [5, 50]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
disaster_response_model = 'DisasterResponseModel.pkl'
pickle.dump(cv, open(disaster_response_model, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.